# Initialization

In [1]:
import numpy as np
from pprint import pprint


from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
#from laboneq.contrib.example_helpers.generate_device_setup import generate_device_setup
from laboneq.dsl.device import DeviceSetup




descriptor = generate_descriptor(
    #pqsc=[""], # 장비 여러개 사용시
    shfqc_6=["DEV12576"],
    number_data_qubits=5,
    multiplex=True,
    number_multiplex=5,
    include_cr_lines=True,
    include_ef_lines=True,
    get_zsync=False,  # Only set to True when using real device
    save = True,
    filename="BF7_KRISS",
    ip_address="10.172.11.217"
)


#descriptor
setup = DeviceSetup.from_descriptor(yaml_text=descriptor, server_host="10.172.11.217")
#setup
setup.instruments[0].device_options = 'SHFQC/PLUS/QC6CH'
#setup.instruments

In [2]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=True, do_emulation=False)
#session.disconnect()

[2025.09.11 20:37:21.884] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /home/kriss/JSAHN/qubit-experiment/examples/laboneq_output/log
[2025.09.11 20:37:21.885] INFO    VERSION: laboneq 2.58.0
[2025.09.11 20:37:21.886] INFO    Connecting to data server at 10.172.11.217:8004
[2025.09.11 20:37:21.893] INFO    Connected to Zurich Instruments LabOne Data Server version 25.07.0.507 at 10.172.11.217:8004
[2025.09.11 20:37:21.912] INFO    Configuring the device setup
[2025.09.11 20:37:21.921] INFO    The device setup is configured


In [3]:
from qpu_types.transmon import TransmonQubit, TransmonQubitParameters
from helper import load_qubit_parameters, save_qubit_parameters
from operations import TransmonOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform



# _loaded_qubits = load_qubit_parameters(filename="latest") 
# qubits = list(_loaded_qubits.values())

qubits = TransmonQubit.from_device_setup(setup) # Also can be done with load_qubit_parameters 




qpu = QPU(quantum_elements=qubits, quantum_operations=TransmonOperations())
#quantum_platform = QuantumPlatform(setup=setup, qpu=qpu)

# qpu.topology.add_edge("bus", "q16", "q17")
# qpu.topology.add_edge("bus", "q17", "q18")
# qpu.topology.add_edge("bus", "q18", "q19")
# qpu.topology.plot()




from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
workflow.logbook.LoggingStore().activate()
#workflow.logbook.LogbookStore().deactivate()


BF 7

In [ ]:
#0 : 15 , 1 : 16 , 2 : 17, 3 : 18, 4 : 19
for q in qubits:
    #q.parameters.ge_drive_pulse["sigma"] = 0.25
    q.parameters.readout_amplitude = 1.0
    q.parameters.reset_delay_length = 20e-6

    q.parameters.readout_range_out = -25
    q.parameters.readout_range_in = 0
    q.parameters.drive_range = -5

    q.parameters.readout_lo_frequency = 7.0e9
    q.parameters.drive_lo_frequency = 4.8e9

    q.parameters.spectroscopy_length = 10e-6

    q.parameters.ge_drive_length =64e-9

    q.parameters.reset_delay_length = 60e-6*5


for q in qubits:
    q.parameters.reset_delay_length = 60e-6*5



# qubits[2].parameters.resonance_frequency_ge = 4.716e9 #Q17
# qubits[3].parameters.resonance_frequency_ge = 4.664e9 #Q18
# qubits[4].parameters.resonance_frequency_ge = 4.779e9 #Q19

# qubits[2].parameters.readout_resonator_frequency = 6.922557e9 #R17
# qubits[3].parameters.readout_resonator_frequency = 6.8433e9 #R18
# qubits[4].parameters.readout_resonator_frequency = 6.7739e9 #R19

# for idx, readout_amp in enumerate([1.0,1.0,1.0,1.0,1.0]):
#     qubits[idx].parameters.readout_amplitude = readout_amp



# 1. Spectroscopy

In [ ]:
from helper import adjust_amplitude_for_output_range, calculate_power
adjust_amplitude_for_output_range(initial_output_dbm=0, initial_amplitude= 0.002, new_output_dbm=-5)
calculate_power(output_dbm=0, amplitude=0.001,attenuation=[])

### 1.1 Resonator spectroscopy with amplitude sweep

STRATEGY
1. CW spectroscpy for wide range scan
    * CW spectroscopy can be done for wide range scan 
    * amplitude, frequency 2D sweep 을 통해  power에 따른 broadening 및 SNR 경향 관찰
    * $\Delta F$ : frequency span,  $\Delta f_r$
    * Pick $\Delta F / \Delta f_r$ 1~2

In [ ]:
from experiments import resonator_spectroscopy_amplitude


####################################################################################
q=qubits[2]
q.parameters.readout_amplitude =1.0

pprint(q.readout_parameters())
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-10e6,10e6,501)
amplitudes = np.linspace(0.0, 1.0, 21)
#####################################################################################
options = resonator_spectroscopy_amplitude.experiment_workflow.options()
#workflow.show_fields(options)
options.update(False)
options.use_cw(False)
options.do_analysis(True)
######################################################################################
# Build and run resonator_spectroscopy_amplitude wormflow
res_spec_amp = resonator_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)
res_spec_amp_result=res_spec_amp.run()


### 1.2 Resonator Spectroscopy
* CW not recommended 

In [ ]:
from experiments import resonator_spectroscopy

####################################################################################
q=qubits[2]
q.parameters.readout_amplitude = 0.4
print(q.readout_parameters())
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#frequencies = q.parameters.readout_resonator_frequency + np.linspace(-10e6,10e6,501)
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-15e6,15e6,5001)
#####################################################################################
options = resonator_spectroscopy.experiment_workflow.options()
#workflow.show_fields(options)
options.count(1024*2)
options.update(False)
options.use_cw(False) #qop.measure 
options.fit_complex_resonator(False)
options.fit_lorentzian(True)
######################################################################################
# Build and run resonator_spectroscopy_amplitude wormflow
res_spec = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    options=options
)
res_spec_result = res_spec.run()
res_spec_analysis_result = res_spec_result.tasks["analysis_workflow"]
pprint(res_spec_analysis_result.output)

In [ ]:
res_spec_result.tasks["analysis_workflow"].tasks["fit_data"].output

In [ ]:
# res_spec_result.tasks["analysis_workflow"].tasks["plot_real_imaginary"].output

In [ ]:
from laboneq.simple import show_pulse_sheet
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

#show_pulse_sheet(compiled_experiment=res_spec_result.tasks["compile_experiment"].output,name='test', interactive=True)
#plot_simulation(compiled_experiment=res_spec_result.tasks["compile_experiment"].output)

### 2.1 Qubit Spectroscopy with Amplitude Sweep
* Current qubit spectroscopy modules do not support CW 
* Recommended to use long enough spectroscopy drive (pulsed) to see steady state signals

In [ ]:
from experiments import qubit_spectroscopy_amplitude

###############################################################
q = qubits[2]
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-2e6, 2e6, 101)
amplitudes = np.linspace(0.01, 0.002, 11)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")

pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = qubit_spectroscopy_amplitude.experiment_workflow.options()
options.update(False)
# workflow.show_fields(options)
###################################################################
qubit_spec_amp = qubit_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)

qubit_spec_amp_result = qubit_spec_amp.run()
qubit_spec_amp_result

### 2.2 Qubit Spectroscopy 


In [46]:
from experiments import qubit_spectroscopy

qubits[3].parameters.resonance_frequency_ge = 4.6654e9 #Q18
###############################################################
q = qubits[3]
q.parameters.spectroscopy_amplitude = 0.001
q.parameters.readout_amplitude = 1.0
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-2e6, 2e6, 101)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = qubit_spectroscopy.experiment_workflow.options()
options.update(False)
# workflow.show_fields(options)
###################################################################
qubit_spec = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    options=options
)

qubit_spec_result = qubit_spec.run()
qubit_spec_result

DRIVE range (dBm) : -5, READOUT range (dBm) : -25
('measure',
 {'amplitude': 1.0,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
('drive',
 {'amplitude': 0.001,
  'length': 1e-05,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2025.09.11 21:18:08.791] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 21:18:08.792] INFO      Workflow 'qubit_spectroscopy': execution started at 2025-09-11                
[2025.09.11 21:18:08.792] INFO      12:18:08.791680Z                                                              
[2025.09.11 21:18:08.793] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 21:18:08.802] INFO    Task 'temporary_qpu': started at 2025-09-11 12:18:08.802384Z
[2025.09.11 21:18:08.803] INFO    Task 'temporary_qpu': ended at 2025-0

WorkflowResult(name=qubit_spectroscopy, index=())

In [ ]:
qubits[2].parameters.readout_resonator_frequency


qubits[2].parameters.resonance_frequency_ge = 4.7161

# 2. Rabi

## 2.1 Rabi Amplitude 

In [87]:
from laboneq_applications.experiments import amplitude_rabi
q = qubits[2]
#only at the initial
#q.parameters.ge_drive_amplitude_pi =1.0
#q.parameters.ge_drive_amplitude_pi2=1.0
#######################################################################
amplitudes = np.linspace(0.0, 1.0,101) 
pprint(q.transition_parameters())
print(q.parameters.ge_drive_amplitude_pi)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = amplitude_rabi.experiment_workflow.options()
options.update(True)
options.use_cal_traces(True)


#print(workflow.show_fields(options))

###################################################################
rabi_amp = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    amplitudes=amplitudes,
    options=options
)

rabi_amp_result = rabi_amp.run()
print(rabi_amp_result.tasks['analysis_workflow'].output)

('drive',
 {'amplitude_pi': 0.45887698306087477,
  'amplitude_pi2': 0.22999619718589515,
  'length': 6.4e-08,
  'pulse': {'beta': 0, 'function': 'drag', 'sigma': 0.25}})
0.45887698306087477
DRIVE range (dBm) : -5, READOUT range (dBm) : -25
[2025.09.11 22:06:33.157] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 22:06:33.157] INFO      Workflow 'amplitude_rabi': execution started at 2025-09-11 13:06:33.156691Z   
[2025.09.11 22:06:33.158] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 22:06:33.170] INFO    Task 'temporary_qpu': started at 2025-09-11 13:06:33.170608Z
[2025.09.11 22:06:33.171] INFO    Task 'temporary_qpu': ended at 2025-09-11 13:06:33.171701Z
[2025.09.11 22:06:33.174] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-11 
[2025.09.11 22:06:33.174] INFO    13:06:33.173993Z
[2025.09.11 22:06:33.175] INFO    Task 'temporary_quantum_elements_from_qpu

In [68]:
from laboneq.simple import show_pulse_sheet
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

show_pulse_sheet(compiled_experiment=rabi_amp_result.tasks["compile_experiment"].output,name='test', interactive=True)
#plot_simulation(compiled_experiment=res_spec_result.tasks["compile_experiment"].output)

[2025.09.11 21:28:26.488] INFO    Recompiling the experiment due to missing extra information in the compiled experiment. Compile with `OUTPUT_EXTRAS=True` and `MAX_EVENTS_TO_PUBLISH=1000` to bypass this step with a small impact on the compilation time.
[2025.09.11 21:28:26.490] INFO    Resolved modulation type of oscillator 'q3_readout_acquire_osc' on signal 'q3/acquire' to SOFTWARE
[2025.09.11 21:28:26.491] INFO    Resolved modulation type of oscillator 'q3_drive_ge_osc' on signal 'q3/drive' to HARDWARE
[2025.09.11 21:28:26.491] INFO    Starting LabOne Q Compiler run...
[2025.09.11 21:28:26.520] INFO    Schedule completed. [0.028 s]
[2025.09.11 21:28:26.534] INFO    Code generation completed for all AWGs. [0.013 s]
[2025.09.11 21:28:26.535] INFO    Completed compilation step 1 of 1. [0.043 s]
[2025.09.11 21:28:26.539] INFO    Finished LabOne Q Compiler run.


## 2.2 (optional) Rabi Amplitude Chevron 

In [7]:
from experiments import amplitude_rabi_chevron

#######################################################################
q = qubits[2]
q.parameters.drive_range = 0
amplitudes = np.linspace(0,1.0,21)
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-50e6, 50e6, 201)
pprint(q.transition_parameters())
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = amplitude_rabi_chevron.experiment_workflow.options()

#print(workflow.show_fields(options))

###################################################################
rabi_amp_chev = amplitude_rabi_chevron.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)

rabi_amp_chev_result = rabi_amp_chev.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(rabi_amp_chev_result.tasks['analysis_workflow'].output)


DRIVE range (dBm) : 0, READOUT range (dBm) : -25
[2025.09.11 20:41:50.696] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 20:41:50.697] INFO      Workflow 'amplitude_rabi_chevron': execution started at 2025-09-11            
[2025.09.11 20:41:50.697] INFO      11:41:50.696532Z                                                              
[2025.09.11 20:41:50.697] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 20:41:50.706] INFO    Task 'create_experiment': started at 2025-09-11 11:41:50.706129Z
[2025.09.11 20:41:50.711] INFO    Task 'create_experiment': ended at 2025-09-11 11:41:50.711791Z
[2025.09.11 20:41:50.713] INFO    Task 'compile_experiment': started at 2025-09-11 11:41:50.713107Z
[2025.09.11 20:41:50.715] INFO    Resolved modulation type of oscillator 'q2_readout_acquire_osc' on signal 'q2/acquire' to SOFTWARE
[2025.09.11 20:41:50.715] INFO    Resolved modulation type of 

## 2.3 (optional) Rabi Length
- Not working yet

In [ ]:
# from experiments import time_rabi
# #######################################################################
# q = qubits[2]
# q.parameters.ge_drive_amplitude_pi = 0.3
# lengths = np.linspace(0.0, 240e-9, 21 ),
# print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
# #######################################################################
# options = time_rabi.experiment_workflow.options()

# #print(workflow.show_fields(options))

# ###################################################################
# rabi_length = time_rabi.experiment_workflow(
#     session=session,
#     qpu=qpu,
#     qubits=q,
#     lengths=lengths,
#     options=options
# )

# rabi_length_result = rabi_length.run()
# #qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
# print(rabi_length_result.tasks['analysis_workflow'].output)


## 2.2 (optional) Rabi Length Chevron

In [ ]:
# from experiments import time_rabi_chevron
# #######################################################################
# q = qubits[4]

# frequencies = q.parameters.resonance_frequency_ge + np.linspace(-5e6, 5e6, 21)
# lengths = np.linspace(0.0, 240e-9, 21 ),
# print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
# #######################################################################
# options = time_rabi_chevron.experiment_workflow.options()

# #print(workflow.show_fields(options))

# ###################################################################
# rabi_length_chev = time_rabi_chevron.experiment_workflow(
#     session=session,
#     qpu=qpu,
#     qubits=q,
#     frequencies=frequencies,
#     lengths=lengths,
#     options=options
# )

# rabi_length_chev_result = rabi_length_chev.run()
# #qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
# print(rabi_length_chev_result.tasks['analysis_workflow'].output)


# 3. Ramsey

In [84]:
from experiments import ramsey

q = qubits[2]
#######################################################################
delays = np.linspace(0,20e-6,301)
detunings = 0.4e6
pprint(q.readout_parameters())
#######################################################################
options = ramsey.experiment_workflow.options()
options.update(True)
options.count(1024)
options.use_cal_traces(True)
# Build and run Ramsey workflow (no explicit command table options required)
ramsey_wf = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    options=options
)
ramsey_result = ramsey_wf.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(ramsey_result.tasks['analysis_workflow'].output)

('measure',
 {'amplitude': 1.0,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2025.09.11 22:01:18.628] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 22:01:18.628] INFO      Workflow 'ramsey': execution started at 2025-09-11 13:01:18.627868Z           
[2025.09.11 22:01:18.629] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 22:01:18.640] INFO    Task 'temporary_qpu': started at 2025-09-11 13:01:18.640399Z
[2025.09.11 22:01:18.641] INFO    Task 'temporary_qpu': ended at 2025-09-11 13:01:18.641408Z
[2025.09.11 22:01:18.643] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-11 
[2025.09.11 22:01:18.643] INFO    13:01:18.643585Z
[2025.09.11 22:01:18.644] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-11 13:01:18.644424Z
[2025.09.11 22:01:18.645] INFO    Task

# X180

In [89]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[2]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x180.options()
options.update(False)
options.use_cal_traces(True)
#options.do_pca(False)

#print(workflow.show_fields(options))

###################################################################
error_amp = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_result = error_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_result.tasks['analysis_workflow'].output)

[2025.09.11 23:14:44.093] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 23:14:44.094] INFO      Workflow 'experiment_workflow_x180': execution started at 2025-09-11          
[2025.09.11 23:14:44.094] INFO      14:14:44.093523Z                                                              
[2025.09.11 23:14:44.095] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 23:14:44.098] WARNING Type <class 'dict'> has unsupported keys or values. Keys must be strings and values must have one of the following types: [NoneType, int, float, complex, bool, str, dict, list, tuple, integer, ndarray] [name: experiment_workflow_x180.input.temporary_parameters].
[2025.09.11 23:14:44.105] INFO    Task 'temporary_qpu': started at 2025-09-11 14:14:44.105742Z
[2025.09.11 23:14:44.106] WARNING Type <class 'dict'> has unsupported keys or values. Keys must be strings and values must have one of the following

# X90

In [91]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[2]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x90.options()
options.update(False)
options.use_cal_traces(True)
#options.do_pca(False)

#print(workflow.show_fields(options))

###################################################################
error_amp = amplitude_fine.experiment_workflow_x90(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_result = error_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_result.tasks['analysis_workflow'].output)

[2025.09.11 23:16:48.711] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 23:16:48.712] INFO      Workflow 'experiment_workflow_x90': execution started at 2025-09-11           
[2025.09.11 23:16:48.712] INFO      14:16:48.711678Z                                                              
[2025.09.11 23:16:48.712] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 23:16:48.715] WARNING Type <class 'dict'> has unsupported keys or values. Keys must be strings and values must have one of the following types: [NoneType, int, float, complex, bool, str, dict, list, tuple, integer, ndarray] [name: experiment_workflow_x90.input.temporary_parameters].
[2025.09.11 23:16:48.723] INFO    Task 'temporary_qpu': started at 2025-09-11 14:16:48.722925Z
[2025.09.11 23:16:48.723] WARNING Type <class 'dict'> has unsupported keys or values. Keys must be strings and values must have one of the following 

# Basic Readout optimization
Here we optimize SNR via

* `readout_length`
* `readout-amplitude`

* `readout_integration_length`
* `readout_integration_delay`
* `readout_integration_weight`

In [60]:
qubits[3].parameters.ge_drive_amplitude_pi=0.2517935676011813 
#qubits[3].parameters.ge_drive_amplitude_pi=0.0

In [63]:
qubits[3].parameters.readout_amplitude= 0.5

In [65]:
from laboneq_applications.analysis import iq_blobs
from laboneq_applications.experiments import iq_blobs

#######################################################################
q = qubits[3]

print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = iq_blobs.experiment_workflow.options()


#print(workflow.show_fields(options))

###################################################################
iq = iq_blobs.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    options=options,
    states='ge'
)

iq_result = iq.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(iq_result.tasks['analysis_workflow'].output)


DRIVE range (dBm) : -5, READOUT range (dBm) : -25
[2025.09.11 21:26:05.311] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 21:26:05.312] INFO      Workflow 'iq_blobs': execution started at 2025-09-11 12:26:05.311567Z         
[2025.09.11 21:26:05.312] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.11 21:26:05.320] INFO    Task 'temporary_qpu': started at 2025-09-11 12:26:05.320704Z
[2025.09.11 21:26:05.321] INFO    Task 'temporary_qpu': ended at 2025-09-11 12:26:05.321651Z
[2025.09.11 21:26:05.324] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-11 
[2025.09.11 21:26:05.324] INFO    12:26:05.323942Z
[2025.09.11 21:26:05.324] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-11 12:26:05.324700Z
[2025.09.11 21:26:05.325] INFO    Task 'create_experiment': started at 2025-09-11 12:26:05.325476Z
[2025.09.11 21:26:05.330] INFO    Task 'create_expe

In [ ]:
from experiments import dispersive_shift

In [ ]:
from experiments import time_traces

In [ ]:
from experiments import signal_propagation_delay

# Save Qubit Parameters

In [92]:
from helper import load_qubit_parameters, save_qubit_parameters
save_qubit_parameters(qubits={ qubit.uid : qubit for qubit in qubits}, save_folder="./qubit_parameters", filename="KRISS_q1718_cal")

Qubit parameters stored as JSON in /home/kriss/JSAHN/qubit-experiment/examples/qubit_parameters/20250911-2317_KRISS_q1718_cal.json
